# Political Corruption Article Translation and Classification Pipeline

This notebook performs a complete pipeline for analyzing political corruption articles:

1. **Load and translate** human-annotated news articles from the original language into the target language.
2. **Save** the translated dataset   for further use.
3. **Classify** the translated
4.  articles using a large language model (LLM)-based classifier (`classify_article`), extracting evidence, rationale, confidence scores, and tentative labels.
5. **Save** the classification results as both CSV and Excel files for easy sharing and review.
6. **Evaluate** the classification performance by comparing the LLM-generated labels against the human-annotated ground truth using a classification report.

This workflow streamlines the process from raw human annotations to translated, LLM-annotated, and evaluated datasets ready for analysis or reporting.


In [ ]:
import os
import pandas as pd
from dataloader import load_and_prepare_data, balanced_sample, load_human_annotated_for_translation
from translation import translate_dataframe
from utils.classifier import classify_article
from sklearn.metrics import classification_report

# Step 1: Load and translate human-annotated data
df_anno = load_human_annotated_for_translation()
df_anno["text"] = df_anno["combined_text"]
df_translated_anno = translate_dataframe(df_anno)

# Prepare output dataframe with selected columns
df = pd.DataFrame({
    "original_text": df_anno["combined_text"],
    "translated_text": df_translated_anno["translated_text"],
    "uri": df_anno.get("uri", ""),  # Use empty string if 'uri' missing
    "corruption_label_m": df_anno["corruption_label_m"]
})

# Save translated annotations CSV
translated_csv_path = os.path.expanduser(
    '~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/annotations/classified_pol_corruption_gabriele_translated.csv'
)
df.to_csv(translated_csv_path, index=False)
print(f"Translated annotations saved to: {translated_csv_path}")

# Step 2: Classify translated articles using the LLM classifier
results = {
    "llm_evidence": [],
    "llm_rationale": [],
    "llm_confidence": [],
    "llm_label": []
}

print(f"Processing {len(df)} articles for classification...")

for idx, row in df.iterrows():
    print(f"Classifying article {idx + 1}/{len(df)}")
    article_text = row["translated_text"]

    if not isinstance(article_text, str) or len(article_text.strip()) == 0:
        print("⚠️ Skipping empty article.")
        results["llm_evidence"].append("")
        results["llm_rationale"].append("No content")
        results["llm_confidence"].append(None)
        results["llm_label"].append("No")
        continue

    output = classify_article(article_text)

    evidence = "; ".join(output.get("highlights", []))
    results["llm_evidence"].append(evidence)
    results["llm_rationale"].append(output.get("rationale", ""))
    results["llm_confidence"].append(output.get("confidence", ""))
    results["llm_label"].append(output.get("tentative_label", ""))

# Add classification results to DataFrame
df["llm_evidence"] = results["llm_evidence"]
df["llm_rationale"] = results["llm_rationale"]
df["llm_confidence"] = results["llm_confidence"]
df["llm_label"] = results["llm_label"]

# Step 3: Save final annotated DataFrame as CSV and Excel
ANNOTATION_PATH = os.path.expanduser('~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/annotations/')
OUTPUT_FILE = "df_output_with_llm_annotations.csv"
os.makedirs(ANNOTATION_PATH, exist_ok=True)

csv_path = os.path.join(ANNOTATION_PATH, OUTPUT_FILE)
xlsx_path = csv_path.replace('.csv', '.xlsx')

df.to_csv(csv_path, index=False)
print(f"CSV saved to: {csv_path}")

df.to_excel(xlsx_path, index=False)
print(f"Excel saved to: {xlsx_path}")

# Step 4: Evaluate classification with sklearn classification_report
# Map ground truth to "Yes" or "No"
y_true = df["corruption_label_m"].map({
    "political corruption": "Yes",
    "no political corruption": "No"
})

# Map predictions, collapsing 'Unsure' and 'Mentioned but not central' into 'No'
def map_prediction(label):
    if not isinstance(label, str):
        return "No"
    label = label.strip().capitalize()
    if label == "Yes":
        return "Yes"
    elif label in ["No", "Unsure", "Mentioned but not central"]:
        return "No"
    else:
        return "No"

y_pred = df["llm_label"].map(map_prediction)

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, labels=["Yes", "No"], zero_division=0))


🌍 Starting translation for multilingual dataset using 'combined_text'


🔁 Translating:   0%|          | 0/201 [00:00<?, ?it/s]

Translating chunk 1/1 (chars: 1152)


🔁 Translating:   1%|          | 2/201 [00:04<07:58,  2.40s/it]

Translating chunk 1/1 (chars: 837)


...

Translating chunk 1/1 (chars: 1244)


🔁 Translating:  45%|████▌     | 91/201 [10:16<13:29,  7.36s/it]

Translating chunk 1/1 (chars: 1292)


🔁 Translating:  46%|████▌     | 92/201 [10:19<10:56,  6.02s/it]

Translating chunk 1/5 (chars: 1464)
Translating chunk 2/5 (chars: 1492)
